In [ ]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
df = pd.read_csv("/content/train.tsv", sep="\t")


In [ ]:
df.head()


,topic_id,initial_request,topic_desc,clarification_need,facet_id,facet_desc,question_id,question,answer
0,1,Tell me about Obama family tree.,Find information on President Barack Obama\'s ...,2,F0001,"Find the TIME magazine photo essay ""Barack Oba...",Q00384,are you interested in seeing barack obamas family,yes am interested in obamas family
1,1,Tell me about Obama family tree.,Find information on President Barack Obama\'s ...,2,F0001,"Find the TIME magazine photo essay ""Barack Oba...",Q03442,would you like to know barack obamas geneology,yes i want to know who made up his family
2,1,Tell me about Obama family tree.,Find information on President Barack Obama\'s ...,2,F0001,"Find the TIME magazine photo essay ""Barack Oba...",Q03402,would you like to know about obamas ancestors,yes this is what am looking for
3,1,Tell me about Obama family tree.,Find information on President Barack Obama\'s ...,2,F0001,"Find the TIME magazine photo essay ""Barack Oba...",Q03725,would you like to know who is currently alive ...,no find the barack obamas family tree time mag...
4,1,Tell me about Obama family tree.,Find information on President Barack Obama\'s ...,2,F0001,"Find the TIME magazine photo essay ""Barack Oba...",Q00769,are you looking for biological information on ...,no i need the specific time magazine article t...


In [ ]:
df.shape


(9176, 9)

In [ ]:
df['initial_request']=df['initial_request'].str.strip()


In [ ]:
df['initial_request'].unique()


array(['Tell me about Obama family tree.', 'What is Fickle Creek Farm',
       'Tell me about sonoma county medical services.',
       'Tell me about of Ralph Owen Brester.',
       "I'm looking for information about mayo clinic Jacksonville FL",
       'How to prepare for the GMAT?',
       "I'd like to learn about lymphoma in dogs",
       'Is Kenmore heater good', 'Tell me more about HP mini 2140',
       'Tell me about the pacific northwest laboratory.',
       'What is California Franchise Tax Board',
       'What are specific dangers of asbestos?',
       'How to write a thank you letter after an interview?',
       'how to hire disc jockey', 'TV on computer',
       'Tell me more about Culpeper National Cemetry',
       'Tell me about Bowflex Power Pro.', 'butter and margarine',
       "I'm looking for information on duchess county tourism",
       "Tell me information about Iowa's food stamp program", 'map',
       'What is equal opportunity employer?',
       "I'm interested i

In [ ]:
df['clarification_need'].values

array([2, 2, 2, ..., 1, 1, 1])

In [ ]:
VALIDATION_SPLIT=0.20


In [ ]:
df = df.drop(['topic_id','topic_desc', 'facet_id','facet_desc','question_id','question','answer'], axis=1)
df.drop_duplicates()
df.head()

,initial_request,clarification_need
0,Tell me about Obama family tree.,2
1,Tell me about Obama family tree.,2
2,Tell me about Obama family tree.,2
3,Tell me about Obama family tree.,2
4,Tell me about Obama family tree.,2


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

In [ ]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH=30

In [ ]:
!pip install pad_sequences

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pad_sequences: filename=pad_sequences-0.6.1-py3-none-any.whl size=10212 sha256=d73e1f307a3b978686f0648e8821aa6601521bc661e52536ca1265333440784b
  Stored in directory: /root/.cache/pip/wheels/c4/d9/67/a54f34d9d990867c70393d7904a47df2f20f3712c1d246d5f5
Successfully built pad_sequences


In [ ]:
from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
from keras.utils import pad_sequences
import re

In [ ]:
data=df.copy()


In [ ]:
print(data['clarification_need'].value_counts())

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, split=' ')
tokenizer.fit_on_texts(data['initial_request'].values)
X = tokenizer.texts_to_sequences(data['initial_request'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)

2    3546
3    3474
4    1480
1     676
Name: clarification_need, dtype: int64


In [ ]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Y = data['clarification_need']
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(Y)
Y=le.transform(Y) 
labels = to_categorical(np.asarray(Y))
print('Shape of data tensor:', X.shape)
print('Shape of label tensor:', labels.shape)


# split the data into a training set and a validation set
indices = np.arange(X.shape[0])
np.random.shuffle(indices)
X = X[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * X.shape[0])

x_train = X[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = X[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]
#y_train = to_categorical(y_train, 4)
#y_val = to_categorical(y_val, 4)

Found 470 unique tokens.
Shape of data tensor: (9176, 30)
Shape of label tensor: (9176, 4)


In [ ]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip
!unzip glove*.zip

--2023-04-02 13:09:19--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2023-04-02 13:09:19--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2023-04-02 13:09:20--  https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]


In [ ]:
!apt install unzip
!unzip "glove.42B.300d.zip"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-25ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
Archive:  glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [ ]:
import os
os.listdir()

['.config',
 'glove.42B.300d.zip',
 'train.tsv',
 'glove.6B.zip',
 'glove.42B.300d.txt',
 'sample_data']

In [ ]:
embeddings_index = {}
f = open('/content/glove.42B.300d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 1917494 word vectors.


In [ ]:
EMBEDDING_DIM=300

In [ ]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [ ]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [ ]:
embed_dim = 300
lstm_out = 196

model = Sequential()
model.add(embedding_layer)

model.add(LSTM(lstm_out, recurrent_dropout=0.25))
#model.add(LSTM(lstm_out, dropout=0.25, dropout_W=0.25))
model.add(Dense(4,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 30, 300)           141300    
                                                                 
 lstm_12 (LSTM)              (None, 196)               389648    
                                                                 
 dense_13 (Dense)            (None, 4)                 788       
                                                                 
Total params: 531,736
Trainable params: 390,436
Non-trainable params: 141,300
_________________________________________________________________
None


In [ ]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          validation_data=(x_val, y_val))

Epoch 1/20
58/58 [==============================] - 31s 462ms/step - loss: 1.2456 - accuracy: 0.3795 - val_loss: 1.2267 - val_accuracy: 0.3837
Epoch 2/20
58/58 [==============================] - 26s 458ms/step - loss: 1.2168 - accuracy: 0.4070 - val_loss: 1.2345 - val_accuracy: 0.3858
Epoch 3/20
58/58 [==============================] - 27s 475ms/step - loss: 1.2121 - accuracy: 0.4093 - val_loss: 1.2317 - val_accuracy: 0.3853
Epoch 4/20
58/58 [==============================] - 28s 476ms/step - loss: 1.2069 - accuracy: 0.4211 - val_loss: 1.2374 - val_accuracy: 0.3831
Epoch 5/20
58/58 [==============================] - 26s 458ms/step - loss: 1.2052 - accuracy: 0.4174 - val_loss: 1.2369 - val_accuracy: 0.3755
Epoch 6/20
58/58 [==============================] - 27s 475ms/step - loss: 1.2032 - accuracy: 0.4141 - val_loss: 1.2390 - val_accuracy: 0.3842
Epoch 7/20
58/58 [==============================] - 28s 478ms/step - loss: 1.2027 - accuracy: 0.4166 - val_loss: 1.2435 - val_accuracy: 0.3940

In [ ]:
example = tokenizer.texts_to_sequences(["Tell me about Obama family tree."])
example = pad_sequences(example, maxlen=MAX_SEQUENCE_LENGTH)

p = model.predict(example)
print(p)

1/1 [==============================] - 0s 43ms/step
[[0.04716685 0.46485087 0.36567566 0.12230655]]
